In [1]:
import sqlite3
import pandas as pd

Найденный датасет представляет собой реальный набор данных от образовательной компании. У них есть своя платформа, где каждый студент может проверить правильность своего решения и получить другую обратную связь. Таблица checker хранит журналы регистрации того, когда и какие лабораторки проверяли пользователи. Компания решила создать новую страницу на платформе, Newsfeed, где эти журналы видны всем студентам программы. Журналы посещений страницы хранятся в другой таблице - pageviews. Гипотеза заключалась в том, что эта страница создаст давление со стороны сверстников, и студенты начнут работать над лабораторными работами раньше. Это может быть хорошо, потому что они смогут сделать больше итераций и попробовать разные подходы. В этой серии упражнений вы попытаетесь выяснить, верна ли эта гипотеза.

In [2]:
con = sqlite3.connect("data/checking-logs.sqlite.sqlite_copy")

<h2>Таблица Pageviews</h2>

In [3]:
pd.read_sql('PRAGMA table_info(pageviews);', con)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,uid,TEXT,0,None,0
2,2,datetime,TIMESTAMP,0,None,0


Таблица pageviews состоит из столбца с числовым нарастающим индексом (не интересует), uid - идентификатор пользователя, datetime - дата посещения новостной платформы. Эти поля нам пригодятся для анализа. Пример данных

In [4]:
pd.read_sql('SELECT * FROM pageviews LIMIT 10;', con)

,index,uid,datetime
0,0,admin_1,2020-04-17 12:01:08.463179
1,1,admin_1,2020-04-17 12:01:23.743946
2,2,admin_3,2020-04-17 12:17:39.287778
3,3,admin_3,2020-04-17 12:17:40.001768
4,4,admin_1,2020-04-17 12:27:30.646665
5,5,admin_1,2020-04-17 12:35:44.884757
6,6,admin_1,2020-04-17 12:35:52.735016
7,7,admin_3,2020-04-17 12:36:21.401412
8,8,admin_3,2020-04-17 12:36:22.023355
9,9,admin_1,2020-04-17 13:55:19.129243


<h2>Таблица Checker</h2>


In [5]:
pd.read_sql('PRAGMA table_info(checker);', con)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,status,TEXT,0,None,0
2,2,success,INTEGER,0,None,0
3,3,timestamp,TIMESTAMP,0,None,0
4,4,numTrials,INTEGER,0,None,0
5,5,labname,TEXT,0,None,0
6,6,uid,TEXT,0,None,0


Таблица **checker** состоит из колонок: *index* - числовой нарастающий номер строки (не интересует), *status* - статус лабараторки (checking, ready), *success* - показатель успешно выполненной работы, *timestamp* - дата записи лога, *numTrials* - количество попыток, *labname* - название лабки, *uid* - идентификатор пользователя. Пример данных

In [6]:
pd.read_sql('SELECT * FROM checker LIMIT 10;', con)

,index,status,success,timestamp,numTrials,labname,uid
0,0,checking,0,2020-04-16 21:12:50.740474,5,None,admin_1
1,1,ready,0,2020-04-16 21:12:54.708365,5,code_rvw,admin_1
2,2,checking,0,2020-04-16 21:46:47.769088,7,None,admin_1
3,3,ready,0,2020-04-16 21:46:48.121217,7,lab02,admin_1
4,4,checking,0,2020-04-16 21:53:01.862637,6,code_rvw,admin_1
5,5,ready,0,2020-04-16 21:53:05.373389,6,code_rvw,admin_1
6,6,checking,0,2020-04-17 05:18:51.965864,1,None,None
7,7,ready,0,2020-04-17 05:19:02.744528,1,project1,user_4
8,8,checking,0,2020-04-17 05:22:35.249331,2,project1,user_4
9,9,ready,1,2020-04-17 05:22:45.549397,2,project1,user_4


<h2>Таблица deadlines</h2>

In [18]:
pd.read_sql('PRAGMA table_info(deadlines);', con)


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,labs,TEXT,0,None,0
2,2,deadlines,INTEGER,0,None,0


In [19]:
pd.read_sql('SELECT * FROM deadlines LIMIT 10;', con)


,index,labs,deadlines
0,0,laba04,1587945599
1,1,laba04s,1587945599
2,2,laba05,1588550399
3,4,laba06,1590364799
4,5,laba06s,1590364799
5,3,project1,1589673599


Таблица **deadlines** состоит из двух колонок: *labs* - название лабки, *deadlines* - количество секунд с начала эпохи

Для проверки гипотезы мы должны получить соединенные данные по двум таблицам с помощью идентификатора пользователя *uid*. Притом нас будет интересовать как факт посещения новостного сайта именно самая первая дата посещения страницы пользователем, считается именно с этого момента пользователь активно пользуется ресурсом и с этого момента можно считать проанализировать влияние посещения пользователем сайта на сдачу лабок. Относительно данных о сдаче лабок нас интересует только записи о успешно сданных работах поэтому в запросе присутствуют следующие фильтры:
<ul>
    <li>status = 'ready', мы не хотим анализировать журналы, которые находятся в стадии проверки статуса</li>
    <li>numTrials = 1, мы хотим анализировать только первые коммиты, потому что только они могут сказать нам, когда студент начал работать над лабкой</li>
    <li>labnames должны быть из списка: 'laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1'. Только они были активны во время эксперимента (по информации из источника)</li>
</ul>

In [8]:
datamart = pd.read_sql("""
            SELECT c.uid, c.labname, c.timestamp as first_commit_ts, p.d as first_view_ts 
            FROM checker as c
            LEFT JOIN (SELECT uid, min(datetime) as d FROM pageviews GROUP BY uid) as p ON c.uid = p.uid
            WHERE status = 'ready' and numTrials = 1 and c.uid LIKE 'user%'
                and labname in ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1') ;""",
                       con, parse_dates=['first_commit_ts', 'first_view_ts'])

Разделим данные на две группы: пользователи, которые посещали новостной сайт - тест-группа и пользователи, которые не посещали новостной портал - контрольная группа.

In [9]:
test = datamart[~datamart.first_view_ts.isna()]

In [10]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59 entries, 3 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              59 non-null     object        
 1   labname          59 non-null     object        
 2   first_commit_ts  59 non-null     datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 2.3+ KB


In [11]:
control = datamart[datamart.first_view_ts.isna()]

мы хотим проверить гипотезу о том, что пользователи, посетившие ленту новостей всего несколько раз, имеют меньшую разницу между первым коммитом и дедлайном. Для этого необходимо рассчитать коэффициент корреляции между количеством просмотров страниц и разницей. (не берем во внимание лабку 'project1', у нее более длительные сроки и она будет являться выбросом)

In [30]:
views_diff = pd.read_sql("""
                SELECT t.uid, avg((CAST(strftime('%s', first_commit_ts) as integer) - deadlines) / 3600) as avg_diff, count(*) as pageviews
                FROM test as t
                JOIN deadlines as d ON d.labs =  t.labname AND t.labname <> 'project1' 
                JOIN pageviews p ON p.uid = t.uid
                GROUP BY t.uid""", con)

In [31]:
views_diff.sort_values('pageviews')

,uid,avg_diff,pageviews
4,user_18,-5.666667,9
10,user_30,-145.250000,12
6,user_21,-95.500000,40
5,user_19,-98.750000,64
0,user_1,-64.400000,140
3,user_17,-61.600000,235
2,user_14,-159.000000,429
1,user_10,-74.800000,445
8,user_28,-86.400000,745
7,user_25,-92.600000,895


Можно заметить, что с небольшим количеством посещений новостного портала, разница между дедлайном и сдаче работы гораздо меньше, чем с большим числом посещений страницы. Далее посчитаем корреляционный коэффициент.

In [32]:
views_diff.corr(numeric_only=True)

,avg_diff,pageviews
avg_diff,1.000000,-0.185834
pageviews,-0.185834,1.000000


Значение имеют слабую обратную корреляцию (обратная в связи с отрицательным знаком разницы) - значит есть зависимость между количеством просмотров 
новостной страницы и разницей между сдачей работы и дедлайном.

Итак... наконец выясним, повлияла ли лента новостей на поведение студентов. Начали ли они работать над лабораторными работами раньше? </br>
Пора поставить гипотезу, которую мы хотим проверить. Разница между первым коммитом и дедлайном вырастет, если человек посмотрит новостную страницу до коммита - положительное влияние посещения новостного сайта.</br>
Помните, что у нас есть две подготовленные таблицы в базе данных **test** и **control**. Проведем нечто похожее на A/B-тест. Нам нужно подсчитать, какой была дельта между первым коммитом и дедлайном до того, как они впервые посетили страницу, и после. То же самое нужно сделать и для контрольной группы. Другими словами, у каждого пользователя в тесте есть своя временная метка для первого посещения ленты новостей. Мы хотим вычислить среднюю дельту (первый коммит - дедлайн) до этой временной метки и после нее. То же самое мы сделаем для пользователей из контрольной группы. Но из-за того, что они вообще не посещали ленту новостей, я решил использовать среднюю временную метку первого просмотра из тестовой группы для пользователей из контрольной группы. Если дельта до первого посещения Ленты новостей значительно отличается от дельты после в тестовой группе, и мы не видим такого же эффекта в контрольной группе, то создание страницы было отличной идеей. Мы можем распространить ее на всю группу.

In [33]:
control.fillna({'first_view_ts': test.first_view_ts.mean()}, inplace=True)

C:\Users\Ivan\AppData\Local\Temp\ipykernel_5440\4099935974.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control.fillna({'first_view_ts': test.first_view_ts.mean()}, inplace=True)


In [39]:
control.to_sql(name='control', con=con, index=False)
test.to_sql(name='test', con=con, index=False)

59

In [41]:
test_results = pd.read_sql("""select time, avg(diff) avg_diff from (
                select 
                    case when first_commit_ts <= first_view_ts then 'before' else 'after' end as time,
                    (CAST(strftime('%s', first_commit_ts) as integer) - deadlines) / 3600 as diff
                from test t
                join deadlines d on t.labname = d.labs and t.labname <> 'project1') t
                group by time                
                    """, con)

In [43]:
test_results

,time,avg_diff
0,after,-103.40625
1,before,-60.56250


Заметим большую разницу в значения разности между событиями До просмотра страницы и После - говорит о верности поставленной гипотезы H1.

In [42]:
control_results = pd.read_sql("""select time, avg(diff) avg_diff from (
                select 
                    case when first_commit_ts <= first_view_ts then 'before' else 'after' end as time,
                    (CAST(strftime('%s', first_commit_ts) as integer) - deadlines) / 3600 as diff
                from control t
                join deadlines d on t.labname = d.labs and t.labname <> 'project1') t
                group by time                
                    """, con)

In [44]:
control_results

,time,avg_diff
0,after,-112.710526
1,before,-99.464286


Видим, что противоположная гипотеза, в этом случае тоже верна, для людей не посещавщих страницы особой разницы в значениях не замечено.

Решение выполнено через базы данных, так как подобранный мной датасет представляет собой базу данных и ресурс из которого я его взял подразумевал работу с базой данных (источник School 21 ссылка <a href="https://drive.google.com/file/d/1zQ8AR2Ry3ajzB3UZO1Sfk3xtDJlzQF2M/view">ТЫК</a>. Также мне удобно было решить задачу в таком формате, так как я SQL-разработчик и в базе работать удобнее, пока мы не изучили инструмента удобнее.